'''
Program to read the input data and process it for the transformer model
- Read the file and split into english and corresponding python code
- format the python code into one line
    - factor new line
    - factor tabs

Also build a formatter that converts the single line python code
back to proper syntax for python. This will used for visualization
and verifing the output. This will be used to evaluate the model.


Challenges in the input data

    - Messy start various options to segment the input data 
        #write ; #1 ; #  write
    - clean input file
        - removed # comments len < 10 & 20 after checking 
        - updated few manually
        - remove comments in the code ???
        # dd - remove pattern

Segment the input file
    identify marker - messy data
    extract segment
    extract english sentence within segment
    extract python code within python
    format python code to one line




'''

In [14]:
'''
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en
'''

'\nimport sys\n!{sys.executable} -m pip install spacy\n!{sys.executable} -m spacy download en\n'

In [72]:
#!pip install SpaCy
#!pip install spacy-wordnet
#!pip install PyDictionary
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/senthilvinayagam/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [16]:
import os
import re

import pandas as pd
import spacy
import random

In [17]:
print(os.listdir())

['clean_data.txt', 'data_processing-v1.ipynb', 'data_processing.ipynb', 'out', 'p_data.csv', 'english_python_data.txt', 'analysis_shortv3.txt', 'analysis_shortv2.txt', 'data_preparation.ipynb', 'clean_in10.txt', 'data.csv', 'analysis.txt', 'clean_in10_20.txt', '.ipynb_checkpoints', 'analysis_short10.txt', 'analysis_short.txt']


In [18]:
#out dataframe
out_df = pd.DataFrame(columns = ['src', 'python'])

In [19]:
input_file = 'clean_data.txt'
output_file = os.path.join('analysis_shortv3.txt')
# first print all lines starting with # with count
with open(output_file,'w') as out_file:
    eng_buf = ''
    py_buf = ''
    samples = 0
    with open(input_file) as in_file:
        for idx,line in enumerate(in_file):
            if line.startswith('#') and len(line) > 30:
                samples = samples + 1
                out_df.loc[len(out_df)] = [eng_buf,py_buf]
                py_buf = ''
                eng_buf = line
                #print(line)
                #out_file.write(line)
            else :
                py_buf = py_buf + line
                #out_file.write(line)
            #if samples > 10:
                #break
print(f" Count of # :{idx}")

 Count of # :42424


In [20]:
len(out_df['python'])

4428

In [21]:
# Replace tabs with 4 spaces
out_df['python']= out_df['python'].str.replace('\t', '    ')

In [22]:
# clean spaces 
# 3->4
# 7->8
# 11->12

reg3s_pat = re.compile(r'(:?\n)[\s]{3}([\w])')
reg7s_pat = re.compile(r'(:?\n)[\s]{7}([\w])')
reg11s_pat = re.compile(r'(:?\n)[\s]{11}([\w])')

def regex_clean(val):

    clean_py = reg3s_pat.sub(r'\1    \2', val)
    clean_py = reg7s_pat.sub(r'\1        \2', clean_py)
    clean_py = reg11s_pat.sub(r'\1            \2', clean_py)
    
    return clean_py

out_df['trg'] = out_df['python'].apply(regex_clean)

In [23]:
spacy_en = spacy.load('en_core_web_sm')

In [33]:
out_df.head()

,src,python,trg
0,,,
1,# write a python program to add two numbers\n,num1 = 1.5\nnum2 = 6.3\nsum = num1 + num2\npri...,num1 = 1.5\nnum2 = 6.3\nsum = num1 + num2\npri...
2,# write a python function to add two user prov...,"def add_two_numbers(num1, num2):\n sum = nu...","def add_two_numbers(num1, num2):\n sum = nu..."
3,# write a program to find and print the larges...,\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 >=...,\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 >=...
4,# write a program to find and print the smalle...,num1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 <= n...,num1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 <= n...


In [98]:
idx = random.randint(0,len(out_df))

print(out_df['src'].iloc[idx])
print(out_df['trg'].iloc[idx])
src_val = out_df['src'].iloc[idx]

# write a python program to rotate a list 10 times and print it

list = [11,22,33,44,55,66,77,88,99]
n = 10
finalList = []
for i in range(0, N):
    finalList.append(list[(i+d)%N])
print(finalList)






In [100]:
doc = spacy_en(src_val)
print(src_val)
for token in doc:
    if token.pos_ == 'VERB':
        print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

# write a python program to rotate a list 10 times and print it

write write VERB VB ROOT xxxx True False
rotate rotate VERB VB relcl xxxx True False
print print VERB VB conj xxxx True False


In [118]:
verb_list = []
for idx,row in out_df.iterrows():
    doc = spacy_en(row['src'])
    for token in doc:
        if token.pos_ == 'VERB':
            verb_list.append(token.text.lower())
            


In [123]:
from collections import Counter
Counter(verb_list).most_common(30)

[('write', 3578),
 ('print', 882),
 ('given', 656),
 ('find', 405),
 ('return', 352),
 ('using', 304),
 ('check', 257),
 ('is', 210),
 ('calculate', 197),
 ('convert', 175),
 ('returns', 173),
 ('are', 147),
 ('remove', 137),
 ('get', 113),
 ('takes', 107),
 ('add', 105),
 ('generate', 104),
 ('count', 82),
 ('create', 75),
 ('demonstrate', 67),
 ('provided', 65),
 ('sort', 56),
 ('define', 55),
 ('compute', 52),
 ('accepts', 50),
 ('implement', 48),
 ('prints', 46),
 ('merge', 43),
 ('separated', 42),
 ('based', 41)]

In [ ]:
'multiply' 